## 7324 Assignment A6 : NN and PCA
##### Name: Thang Nguyen
##### SMU ID: 48689334

## Imports

In [1]:
# The Classifiers
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from sklearn.decomposition import PCA
# Data Wrangling Tool
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Foundational libraries
import pandas as pd
import numpy as np

## Loading Dataset

In [2]:
# note: I added the column names manually into the csv prior
# original csv from UCI comes without columns
abalone_df = pd.read_csv("../data/abalone.csv")
# checking dataframe
abalone_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             4177 non-null   object 
 1   Length          4177 non-null   float64
 2   Diameter        4177 non-null   float64
 3   Height          4177 non-null   float64
 4   Whole Weight    4177 non-null   float64
 5   Shucked Weight  4177 non-null   float64
 6   Viscera Weight  4177 non-null   float64
 7   Shell Weight    4177 non-null   float64
 8   Rings           4177 non-null   int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 293.8+ KB


In [3]:
abalone_df.head()

,Sex,Length,Diameter,Height,Whole Weight,Shucked Weight,Viscera Weight,Shell Weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


## Cleaning Dataset

### One-hot Encoding abalone sex

In [4]:
abalone_df = pd.concat([abalone_df, pd.get_dummies(abalone_df['Sex'], prefix = 'Sex')], axis = 1)
abalone_df.drop(['Sex'], axis = 1, inplace = True)

In [5]:
abalone_df.head()

,Length,Diameter,Height,Whole Weight,Shucked Weight,Viscera Weight,Shell Weight,Rings,Sex_F,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,0,0,1
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,0,0,1
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,1,0,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,0,0,1
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,0,1,0


### Categorizing Rings into groups where
#### 0-4 -> infant
#### 5-9 -> young
#### 10-14 -> teen
#### 15-20 -> young adult
#### 20-25 -> adult

In [6]:
def group_rings(rings):
    if rings <= 4:
        return 0
    if rings <= 9:
        return 1
    if rings <= 14:
        return 2
    if rings <= 20: 
        return 3
    if rings <= 25:
        return 4

In [7]:
abalone_df["Rings"] = abalone_df["Rings"].apply(group_rings)

In [8]:
abalone_df.head()

,Length,Diameter,Height,Whole Weight,Shucked Weight,Viscera Weight,Shell Weight,Rings,Sex_F,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,3.0,0,0,1
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,1.0,0,0,1
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,1.0,1,0,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,2.0,0,0,1
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,1.0,0,1,0


## Analysis

### Split and Scale

In [9]:
target = abalone_df["Rings"].values
features = abalone_df.drop('Rings', axis=1)

In [10]:
# scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, target, random_state = 0, train_size = 0.8)

In [12]:
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.int32)

### Building Keras NN

In [13]:
model = Sequential()

#### 1 input/output Layer, inputs equal to feature dimension

In [14]:
model.add(Dense(10, input_dim = X_train.shape[1], activation='tanh'))
model.add(Dense(5, activation='sigmoid'))

In [15]:
# must use categorical and spars
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [16]:
model.fit(X_train, y_train, epochs=500)

Epoch 1/500
  1/105 [..............................] - ETA: 15s - loss: 1.7760 - accuracy: 0.2188

2023-04-13 23:46:38.838172: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


105/105 [==============================] - 0s 473us/step - loss: 1.4030 - accuracy: 0.4837
Epoch 2/500
105/105 [==============================] - 0s 412us/step - loss: 1.0437 - accuracy: 0.6471
Epoch 3/500
105/105 [==============================] - 0s 384us/step - loss: 0.9026 - accuracy: 0.6501
Epoch 4/500
105/105 [==============================] - 0s 381us/step - loss: 0.8254 - accuracy: 0.6549
Epoch 5/500
105/105 [==============================] - 0s 375us/step - loss: 0.7881 - accuracy: 0.6612
Epoch 6/500
105/105 [==============================] - 0s 375us/step - loss: 0.7658 - accuracy: 0.6678
Epoch 7/500
105/105 [==============================] - 0s 387us/step - loss: 0.7504 - accuracy: 0.6708
Epoch 8/500
105/105 [==============================] - 0s 380us/step - loss: 0.7395 - accuracy: 0.6752
Epoch 9/500
105/105 [==============================] - 0s 434us/step - loss: 0.7312 - accuracy: 0.6806
Epoch 10/500
105/105 [==============================] - 0s 379us/step - loss: 0.7233 

#### 1 inner Layer, inputs equal to feature dimension

In [24]:
model.add(Dense(10, input_dim = X_train.shape[1], activation='tanh'))
model.add(Dense(3, activation='tanh'))
model.add(Dense(5, activation='sigmoid'))

In [25]:
# must use categorical and spars
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [26]:
model.fit(X_train, y_train, epochs=500)

Epoch 1/500
105/105 [==============================] - 0s 713us/step - loss: 1.3650 - accuracy: 0.5878
Epoch 2/500
105/105 [==============================] - 0s 578us/step - loss: 1.0534 - accuracy: 0.6833
Epoch 3/500
105/105 [==============================] - 0s 649us/step - loss: 0.9125 - accuracy: 0.6998
Epoch 4/500
105/105 [==============================] - 0s 611us/step - loss: 0.8280 - accuracy: 0.7079
Epoch 5/500
105/105 [==============================] - 0s 615us/step - loss: 0.7678 - accuracy: 0.7103
Epoch 6/500
105/105 [==============================] - 0s 616us/step - loss: 0.7296 - accuracy: 0.7163
Epoch 7/500
105/105 [==============================] - 0s 607us/step - loss: 0.7068 - accuracy: 0.7207
Epoch 8/500
105/105 [==============================] - 0s 716us/step - loss: 0.6923 - accuracy: 0.7186
Epoch 9/500
105/105 [==============================] - 0s 596us/step - loss: 0.6830 - accuracy: 0.7169
Epoch 10/500
105/105 [==============================] - 0s 597us/step - l

### 2 inner Layers, inputs equal to feature dimension

In [20]:
model.add(Dense(10, input_dim = X_train.shape[1], activation='tanh'))
model.add(Dense(3, activation='tanh'))
model.add(Dense(3, activation='tanh'))
model.add(Dense(5, activation='sigmoid'))

In [21]:
# must use categorical and spars
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [22]:
model.fit(X_train, y_train, epochs=500)

Epoch 1/500
105/105 [==============================] - 0s 627us/step - loss: 1.3677 - accuracy: 0.6292
Epoch 2/500
105/105 [==============================] - 0s 464us/step - loss: 1.1107 - accuracy: 0.6702
Epoch 3/500
105/105 [==============================] - 0s 470us/step - loss: 0.9697 - accuracy: 0.6935
Epoch 4/500
105/105 [==============================] - 0s 458us/step - loss: 0.8742 - accuracy: 0.7019
Epoch 5/500
105/105 [==============================] - 0s 466us/step - loss: 0.8153 - accuracy: 0.7031
Epoch 6/500
105/105 [==============================] - 0s 462us/step - loss: 0.7747 - accuracy: 0.7049
Epoch 7/500
105/105 [==============================] - 0s 463us/step - loss: 0.7484 - accuracy: 0.7040
Epoch 8/500
105/105 [==============================] - 0s 464us/step - loss: 0.7298 - accuracy: 0.7049
Epoch 9/500
105/105 [==============================] - 0s 467us/step - loss: 0.7161 - accuracy: 0.7049
Epoch 10/500
105/105 [==============================] - 0s 460us/step - l